# Adamata Test Notebook

## Get python version

In [ ]:
# This version will decide the version I will use
!python -V

## Understanding the datasets a little bit

we're going to download the dataset given from google drive, we just going to use gdown and unzip to download then unzip it.

so...we got 12 images with respective label, and turns out the label is all the same despite the image is different.

```txt
class_id center_x center_y width height
```

if we see from YOLO annotation format, it looks that way, the label is all similar.

In [ ]:
!pip install gdown

## 

## 

In [ ]:
# download and extract given dataset
dataset_url = "https://drive.google.com/file/d/1vbMFTe2E5OHp2h5o_YL-RUoIFKxsrZWT/view?usp=sharing"
dataset_output_path = "/kaggle/working/sample.zip"
!gdown --fuzzy {dataset_url} -O {dataset_output_path}
!unzip {dataset_output_path}

In [ ]:
dataset_directory = "/kaggle/working/sample"

import os
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path

def display_images_from_directory(directory_path):
    """
    Opens a directory, identifies image files, and displays them in a grid.

    Args:
        directory_path (str): The path to the directory containing images.
    """
    image_files = []
    # Iterate through files in the specified directory
    for filename in os.listdir(directory_path):
        # Check if the file is an image (you can extend this list)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            image_files.append(os.path.join(directory_path, filename))

    if not image_files:
        print(f"No image files found in '{directory_path}'")
        return

    # Determine grid size for displaying images
    num_images = len(image_files)
    cols = int(num_images**0.5)  # Approximate square grid
    rows = (num_images + cols - 1) // cols # Calculate rows to fit all images

    plt.figure(figsize=(10, 10)) # Adjust figure size as needed

    for i, image_path in enumerate(image_files):
        try:
            img = Image.open(image_path)
            plt.subplot(rows, cols, i + 1)
            plt.imshow(img)
            plt.title(os.path.basename(image_path)) # Display filename as title
            plt.axis('off') # Hide axes for cleaner display
        except Exception as e:
            print(f"Error opening or displaying '{image_path}': {e}")

    plt.tight_layout() # Adjust subplot parameters for a tight layout
    plt.show()

# Example usage:
# Replace 'path/to/your/images' with the actual path to your directory
display_images_from_directory(dataset_directory)

In [ ]:
def cat_txt_files_in_directory(directory_path):
    """
    Opens a directory and prints the content of all .txt files within it.

    Args:
        directory_path (str): The path to the directory to process.
    """
    path_obj = Path(directory_path)

    if not path_obj.is_dir():
        print(f"Error: '{directory_path}' is not a valid directory.")
        return

    print(f"Contents of .txt files in '{directory_path}':\n")
    found_txt_files = False

    for item in path_obj.iterdir():
        if item.is_file() and item.suffix == '.txt':
            found_txt_files = True
            print(f"--- {item.name} ---")
            try:
                with open(item, 'r', encoding='utf-8') as f:
                    print(f.read())
                print() # Add a newline for better separation between files
            except Exception as e:
                print(f"Error reading '{item.name}': {e}")
    
    if not found_txt_files:
        print("No .txt files found in this directory.")

cat_txt_files_in_directory(dataset_directory)

## Try plot the dataset with the bounding boxes

so the dataset is bunch of image with yolo annotated bounding boxes

lets try to plot it alongside the bounding boxes

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from pathlib import Path

def display_images_with_bboxes(directory_path):
    """
    Display images from a directory with YOLO format bounding boxes.
    
    Args:
        directory_path (str): Path to the directory containing jpg and txt files
    """
    dir_path = Path(directory_path)
    
    # Get all jpg files
    jpg_files = list(dir_path.glob("*.jpg"))
    
    if not jpg_files:
        print(f"No JPG files found in {directory_path}")
        return
    
    # Set up the plot
    fig = plt.figure(figsize=(15, 10))
    
    for i, jpg_path in enumerate(jpg_files):
        # Find corresponding txt file
        txt_path = jpg_path.with_suffix('.txt')
        
        if not txt_path.exists():
            print(f"Warning: No corresponding txt file for {jpg_path.name}")
            continue
            
        # Read image
        image = cv2.imread(str(jpg_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_height, img_width = image.shape[:2]
        
        # Read bounding boxes from txt file
        with open(txt_path, 'r') as f:
            lines = f.readlines()
        
        # Draw bounding boxes
        for line in lines:
            data = line.strip().split()
            if len(data) >= 5:  # class_id, x_center, y_center, width, height
                class_id = int(data[0])
                x_center = float(data[1]) * img_width
                y_center = float(data[2]) * img_height
                width = float(data[3]) * img_width
                height = float(data[4]) * img_height
                
                # Calculate coordinates
                x1 = int(x_center - width/2)
                y1 = int(y_center - height/2)
                x2 = int(x_center + width/2)
                y2 = int(y_center + height/2)
                
                # Draw rectangle
                cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
                
                # Add label with class ID
                label = f'Class {class_id}'
                label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
                
                # Draw label background
                cv2.rectangle(image, (x1, y1 - label_size[1] - 5), 
                             (x1 + label_size[0], y1), (255, 0, 0), -1)
                
                # Draw label text
                cv2.putText(image, label, (x1, y1 - 5), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Display image
        plt.subplot(3, 4, i+1)  # Adjust grid size as needed
        plt.imshow(image)
        plt.title(jpg_path.name)
        plt.axis('off')
        
        # Stop if we've displayed 12 images (adjust as needed)
        if i + 1 >= 12:
            break
    
    plt.tight_layout()
    plt.show()

# Usage example:
display_images_with_bboxes(dataset_directory)

## What we know from the dataset so far

- It hasnt labelled properly
- The bounding boxes is tight, so no need readjustment
- there's only 12 image, so might need augmentation
- it's not square, hate it
- and there's one image that miss one object

What we will do next is using [Roboflow](https://roboflow.com) to adjust the label, adding augmentation and little bit of preprocessing.

imo, it's ideal if we need non technical 

## Dataset augmentation, and using it.

Using Roboflow easy proprocess and augmentation, im able to convert 12 datasets image into 28(limited by Free Plan of Roboflow).

preprocess also squish the image size to 300x300 pixels

For datasets this size its easy to justify using SaaS like it, for the augmentation we pick this:

- Flip: Horizontal, Vertical
- Crop: 0% Minimum Zoom, 15% Maximum Zoom
- Rotation: Between -10° and +10°
- Brightness: Between -15% and +15%
- Blur: Up to 1.2px
- Noise: Up to 0.1% of pixels

This is to represent variation of images with range of realistic error and skewness of the camera.

The datasets also automatically splitted into 3 sets:

- 24 Training images
- 3 Validation images
- 1 Test image

The best part of using Roboflow is we can download the dataset using Curl, easy.

I honestly can do that by myself with little bit of code but Roboflow able to reformat the datasets into multiple annotation formats.

![Roboflow Screenshot](https://github.com/dhupee/adamata-ml-engineer-test/blob/master/img/Screenshot%202025-11-24%20at%2017-01-15%20dataset-annotation%20-%20v3%202-labeled-preprocessed-augmented.png)

## Picking model to be used

For easy life, YOLO is probably the most straight forward model to be used, from [Models from ultralytics](https://docs.ultralytics.com/models/) docs, I decided to use Yolov11n, for smallest model of the newest Yolov11.

In [ ]:
# this dataset isnt inside directory so we need to make new directory
new_dataset_url = "https://app.roboflow.com/ds/Fz6f942UcB?key=qt4eGUWAQp"
new_dataset_path = "/kaggle/working/labelled.zip"
new_dataset_directory = "labelled"
!curl -L {new_dataset_url} > {new_dataset_path}
!mkdir {new_dataset_directory}
!unzip {new_dataset_path} -d {new_dataset_directory}

## Training the YOLO model

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from PIL import Image
import numpy as np

data_path = "/kaggle/working/labelled/data.yaml"
image_size = 300
model = YOLO("yolo11n.pt")
results = model.train(data=data_path, epochs=100, imgsz=image_size)